# Fast initial tests to check good ranges and behaviours
# with printout and visuals

In [ ]:
import os
from functools import partial

import numpy
import scipy
from matplotlib import pyplot

import os

import pyhf
import cabinetry

from pyhf_stuff import (
    fit_cabinetry,
    fit_cabinetry_post,
    fit_linspace,
    fit_normal,
    fit_signal,
    region,
    serial,
    blind,
    fit_mcmc_mix,
    mcmc_core,
)

from pyhf_stuff.region_properties import region_properties
from pyhf_stuff.region_fit import region_fit

In [ ]:
# fill me in
BASEPATH = "/home/tombs/Cambridge/interval-evidence/searches/ins1839446/"

# fits

In [ ]:
def main():
    region_name_to_scan = {
        "SR2JBVEM_meffInc30_gluino": (5, 40),
        "SR2JBVEM_meffInc30_squark": (40, 160),
        "SR4JhighxBVEM_meffInc30": (0, 30),
        "SR4JlowxBVEM_meffInc30": (0, 30),
        "SR6JBVEM_meffInc30_gluino": (0, 12),
        "SR6JBVEM_meffInc30_squark": (0, 12),
    }

    for name, (lo, hi) in region_name_to_scan.items():
        print(name)
        test(name, lo, hi)


def test(name, lo, hi, nbins=10):
    dir_region = os.path.join(BASEPATH, name)
    region_1 = region.Region.load(dir_region)

    dir_fit = os.path.join(dir_region, "fit")

    # cabinetry
    cab = fit_cabinetry.fit(region_1)
    print(cab)
    cab_post = fit_cabinetry_post.fit(region_1)
    print(cab_post)
    
    # normal
    norm = fit_normal.fit(region_1)
    print(norm)
    
    # linspace
    linspace = fit_linspace.fit(region_1, lo, hi, nbins + 1)
    print(linspace)
        
    levels = numpy.array(linspace.levels) - region_fit(region_1).fun
    
    x = numpy.linspace(linspace.start, linspace.stop, len(levels))
    pyplot.plot(x, -levels, "k")
    pyplot.ylim(-8, 0)
    pyplot.show()

    
main()

# mcmc

In [ ]:

def main():
    region_name_to_scan = {
        "SR2JBVEM_meffInc30_gluino": (5, 40),
        "SR2JBVEM_meffInc30_squark": (40, 160),
        "SR4JhighxBVEM_meffInc30": (0, 30),
        "SR4JlowxBVEM_meffInc30": (0, 30),
        "SR6JBVEM_meffInc30_gluino": (0, 12),
        "SR6JBVEM_meffInc30_squark": (0, 12),
    }

    for name, (lo, hi) in region_name_to_scan.items():
        print(name)
        dump_region(name, lo, hi)


def dump_region(name, lo, hi, nbins=50):
    dir_region = os.path.join(BASEPATH, name)
    region_1 = region.Region.load(dir_region)

    dir_fit = os.path.join(dir_region, "fit")

    mix = fit_mcmc_mix.fit(
        region_1,
        nbins,
        (lo, hi),
        seed=0,
        nsamples=10_000,
        nrepeats=8,
        nprocesses=8,
    )
    # mix.dump(dir_fit)

    neff = mcmc_core.n_by_fit(mix).sum()
    nrepeats = mix.nrepeats
    nsamples = mix.nsamples
    total = numpy.sum(mix.yields)
    print(
        "acceptance: %.2f (%d / %d)"
        % (total / (nrepeats * nsamples), total, nrepeats * nsamples)
    )
    print(
        "efficiency: %.2f (%.1f / %.1f)"
        % (nrepeats * neff / total, neff, total / nrepeats)
    )
    
    x = numpy.linspace(*mix.range_, len(mix.yields) + 1)
    y = numpy.array(mix.yields)
    pyplot.hist(
        x[:-1], 
        weights=y / y.max(), 
        range=mix.range_, 
        bins=len(x) - 1,
        histtype="step",
        color="r",
        lw=2,
    )
    
    pyplot.show()

main()

# signal scan

In [ ]:
def main():
    region_name_to_scan = {
        "SR2JBVEM_meffInc30_gluino": (0, 35),
        "SR2JBVEM_meffInc30_squark": (0, 80),
        "SR4JhighxBVEM_meffInc30": (0, 25),
        "SR4JlowxBVEM_meffInc30": (0, 20),
        "SR6JBVEM_meffInc30_gluino": (0, 12),
        "SR6JBVEM_meffInc30_squark": (0, 15),
    }

    for name, (lo, hi) in region_name_to_scan.items():
        print(name)
        dump_region(name, lo, hi)


def dump_region(name, lo, hi, nbins=10):
    dir_region = os.path.join(BASEPATH, name)
    region_1 = region.Region.load(dir_region)

    dir_fit = os.path.join(dir_region, "fit")

    sig = fit_signal.fit(region_1, lo, hi, nbins + 1)
    print(sig)
    
    levels = numpy.array(sig.levels)
    levels -= levels.min()
    
    x = numpy.linspace(sig.start, sig.stop, len(levels))
    pyplot.plot(x, levels, "k")
    pyplot.ylim(0, 8)
    pyplot.show()
    
main()